# 👏🏻欢迎使用E2M(Everything2Markdown)👏🏻

## 📖简介

E2M是一个将各种内容转换为Markdown格式的工具，支持多种输入格式:

-  文本
    -   doc
    -   docx
    -   epub
    -   html
    -   htm
    -   pdf (特别说明:包括纯文本、文本+图片、纯图片形式的pdf文件)
    -   pptx
-   链接
    -   url
-   音频
    -   mp3
    -   m4a
- 视频
    -  mp4 (未完成)


## 🚀快速上手

### 安装

```bash
pip install wisup_e2m
```

### URL -> Markdown

In [1]:
from wisup_e2m import UrlParser

url = "https://www.osar.fr/notes/justintonation"
parser = UrlParser(engine="jina") # url engines: jina

In [2]:
data = parser.parse(url)
print(data.text)

100%|██████████| 1/1 [00:01<00:00,  1.39s/it]

Title: What I Learned Writing an Album in Just Intonation

URL Source: https://www.osar.fr/notes/justintonation

Markdown Content:
Pierre Cusa <pierre@osar.fr>  
August 2024 [(history)](https://github.com/pac-dev/notes/commits/master/content/justintonation.md)  
in [osar.fr](https://www.osar.fr/) / notes

**Just intonation (JI)** today falls under the umbrella of microtonal music, because it involves playing "between the notes" of the 12-tone scale we're used to. As I type microtonal, my spellchecker decorates it with a sad squiggle, confirming I've drifted (again) into a topic that's not even in its dictionary. But JI is often described as _the way music should naturally sound_. So which one is it, an obscure deviation from the norm, or an ideal we should return to?

This post contains all the notes I want to keep after writing [an album](https://www.cusamusic.com/album/walk/) in JI, specifically using a very uncompromising method called freestyle JI. The album itself isn't the focus 

### PDF -> Markdown

In [4]:
# 使用marker引擎会从huggingface hub下载模型，并使用模型进行解析
# 如果遇到网络问题，尝试运行以下代码

import os
os.environ['CURL_CA_BUNDLE'] = ''
os.environ['HF_ENDPOINT']= 'https://hf-mirror.com'

In [5]:
from wisup_e2m import PdfParser

pdf_path = "./Attention is All You Need Paper.pdf"
parser = PdfParser(engine="marker") # pdf engines: marker, unstructured, surya_layout 

Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16
Loaded detection model vikp/surya_layout3 on device mps with dtype torch.float16
Loaded reading order model vikp/surya_order on device mps with dtype torch.float16
Loaded recognition model vikp/surya_rec on device mps with dtype torch.float16


/Users/zephyr/anaconda3/envs/e2m/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded texify model to mps with torch.float16 dtype


/Users/zephyr/anaconda3/envs/e2m/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


In [6]:
data = parser.parse(pdf_path) # 默认会在 ./figure 文件夹下生成图片，可通过参数修改

Finding reading order: 100%|██████████| 2/2 [00:24<00:00, 12.37s/it]


In [7]:
print(data.text)

# Attention Is All You Need

| Ashish Vaswani∗ Google Brain   |                                                 |
|--------------------------------|-------------------------------------------------|
| avaswani@google.com            | Noam Shazeer∗ Google Brain                      |
| noam@google.com                | Niki Parmar∗                                    |
| Google Research                |                                                 |
| nikip@google.com               | Jakob Uszkoreit∗ Google Research usz@google.com |

| Llion Jones∗            |                                         |
|-------------------------|-----------------------------------------|
| Google Research         |                                         |
| llion@google.com        | Aidan N. Gomez∗ † University of Toronto |
| aidan@cs.toronto.edu    | Łukasz Kaiser∗ Google Brain             |
| lukaszkaiser@google.com |                                         |

Illia Polosukhin∗ ‡
illia.polosukhin@gm